In [ ]:
from tqdm import tqdm
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

import nltk
nltk.download('cess_cat')

In [ ]:
from nltk.corpus import cess_cat as corpus

words = []
words_to_remove = ['*0*', '-Fpa-', '-Fpt-']
#for s in tqdm(corpus.sents()[:1000]): # debug or quickly train the network
for s in tqdm(corpus.sents()):
    new_s = ['<s>'] + s[:-1] + ['</s>']
    new_s = [w for w in new_s if w not in words_to_remove]
    words.extend(new_s)


In [ ]:
from torch.utils.data import Dataset, DataLoader

class FixedWindow(Dataset):
    def __init__(self, words, length_window):
        super().__init__()
        self.length_window = length_window
        # TODO:
        # compute the vocabulary = list of unique words in 'words',
        # then assign a unique id number to each word in the
        # vocabulary, 
        # and finally compute a list of ids, one per word in 'words'


    def __len__(self):
        return len(self.ids) - self.length_window

    def __getitem__(self, idx):
        #TODO:
        # returns a pair of tensors (first_ids, last_id) where
        # first_ids are the ids of the words starting at index
        # idx with length length_window-1, and last_id is the
        # id at position idx+self.length_window-1, next to
        # first_ids
        pass



length_window = 5
dataset = FixedWindow(words, length_window)

x, y = dataset.__getitem__(10)
#print('x = {}, y = {}'.format(x, y))

batch_size = 1000 # 5 to debug
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True) # shuffle=False to debug

if True:
    for nbatch, (X, y) in enumerate(dataloader):
        print('batch {}'.format(nbatch))
        print('X = {}'.format(X))
        print('y = {}'.format(y))
        for x,z in zip(X.numpy(), y.numpy()):
            print([dataset.id2word[w] for w in x], end=' ')
            print(dataset.id2word[z])
        if nbatch==3:
            break


In [ ]:
class NNLM(nn.Module):
    def __init__(self, num_classes, dim_input, dim_hidden, dim_embedding):
        super().__init__()
        self.num_classes = num_classes
        self.dim_input = dim_input
        self.dim_hidden = dim_hidden
        self.dim_embedding = dim_embedding
        self.embeddings = nn.Embedding(self.num_classes, self.dim_embedding) # embedding layer or look up table
        self.hidden1 = nn.Linear(self.dim_input * self.dim_embedding, self.dim_hidden, bias=False)
        self.ones = nn.Parameter(torch.ones(self.dim_hidden))       
        self.hidden2 = nn.Linear(self.dim_hidden, self.num_classes, bias=False)
        self.hidden3 = nn.Linear(self.dim_input * self.dim_embedding, self.num_classes, bias=False) # final layer
        self.bias = nn.Parameter(torch.ones(self.num_classes))

    def forward(self, X):
        word_embeds = self.embeddings(X)
        X = word_embeds.view(-1, self.dim_input * self.dim_embedding) # first layer
        tanh = torch.tanh(self.ones + self.hidden1(X)) # tanh layer
        output = self.bias + self.hidden3(X) + self.hidden2(tanh) # summing up all the layers with bias
        return output


num_classes = dataset.vocabulary_size
dim_input = length_window - 1
dim_hidden = 50
dim_embedding = 32
learning_rate= 1e-3
num_epochs = 60

model = NNLM(num_classes, dim_input, dim_hidden, dim_embedding)

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

path = 'NNLM.pt'
do_train = True
do_test = True

In [ ]:
# In the top menu go to Runtime -> Change runtime type and set Hardware 
# accelerator to GPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# Assuming that we are on a CUDA machine, this should print a CUDA device:
print(device)
model = model.to(device)

In [ ]:
from torch.cuda.random import device_count
if do_train:
    size = len(dataloader.dataset)
    for epoch in range(num_epochs):
        for batch, (X, y) in enumerate(dataloader):
            X, y = X.to(device), y.to(device)
            pred = model(X)
            loss = loss_fn(pred, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if batch % 100 == 0:
                loss, current = loss.item(), batch * batch_size
                print('Epoch {} loss: {:>7f}  [{:>5d}/{:>5d}]'
                    .format(epoch+1, loss, current, size))

    torch.save({'model_state_dict': model.state_dict()}, path)
else:
    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])

In [ ]:
if do_test:
    num_sentences = 5
    max_num_words = 100

    nsent = 0
    generated_words = ['<s>', 'El', 'dia', 'que']
    assert len(generated_words)==dim_input # length_window-1

    model.eval()
    with torch.no_grad():
        while (nsent < num_sentences) and (len(generated_words) < max_num_words):
            input_ids = None
            # TODO: set input_ids to the list of ids of the last dim_input generated words

            pred = model(torch.tensor(input_ids).unsqueeze(0).to(device))
            probs = torch.nn.functional.softmax(pred, dim=1)

            #TODO:
            # probs is the probability of each id (word) in the vocabulary.
            # Now you have to select one output_id according to them, either the one
            # with maximum probability, or sample one id according to their
            # distribution in probs.
            # Hint: see doc. on numpy argmax and torch.multinomial
            # What's better ?

            output_word = None
            # TODO: get the word for output_id

            generated_words += [output_word]
            if output_word=='</s>':
                nsent += 1

    generated_text = ' '.join(generated_words)
    generated_text = generated_text.replace(' </s> <s>', '.').replace('<s> ','').replace(' </s>','.')
    for s in [' l\' ',' s\' ',' d\' ',]:
        generated_text = generated_text.replace(s, s[:-1])
    generated_text = generated_text.replace(' , ', ', ').replace('_',' ')
    print(generated_text)